In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [5]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [6]:
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')
poems = gutenberg.raw('blake-poems.txt')
stories = gutenberg.raw('bryant-stories.txt')
busterbrown = gutenberg.raw('burgess-busterbrown.txt')
ball = gutenberg.raw('chesterton-ball.txt')
parents = gutenberg.raw('edgeworth-parents.txt')
mobydick = gutenberg.raw('melville-moby_dick.txt')
paradise = gutenberg.raw('milton-paradise.txt')
emma = gutenberg.raw('austen-emma.txt')
#print(persuasion)
# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
paradise = re.sub(r'Book \w+', '', paradise)
parents = re.sub(r'Book \w+', '', parents)
ball = re.sub(r'[A-Z]{2,100}' , '', ball)
ball = re.sub(r'I\.' , '', ball)
mobydick = re.sub(r'CHAPTER \w+', '', mobydick)
busterbrown = re.sub(r'\w+\t', '', busterbrown)
stories = re.sub(r'[A-Z]{2,100}', '', stories)
poems = re.sub(r'[A-Z]{2,100}', '', poems)
print(ball)

[The Ball and The Cross by G.K. Chesterton 1909]


 A     

The flying ship of Professor Lucifer sang through the skies like
a silver arrow; the bleak white steel of it, gleaming in the
bleak blue emptiness of the evening.  That it was far above the
earth was no expression for it; to the two men in it, it seemed
to be far above the stars.  The professor had himself invented
the flying machine, and had also invented nearly everything in
it.  Every sort of tool or apparatus had, in consequence, to the
full, that fantastic and distorted look which belongs to the
miracles of science.  For the world of science and evolution is
far more nameless and elusive and like a dream than the world of
poetry and religion; since in the latter images and ideas remain
themselves eternally, while it is the whole idea of evolution
that identities melt into each other as they do in a nightmare.

All the tools of Professor Lucifer were the ancient human tools
gone mad, grown into unrecognizable shapes, forge

In [7]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text



    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
emma = text_cleaner(emma)
paradise = text_cleaner(paradise)
parents = text_cleaner(parents)
ball = text_cleaner(ball)
mobydick = text_cleaner(mobydick)
busterbrown = text_cleaner(busterbrown)
stories = text_cleaner(stories)
poems = text_cleaner(poems)
print(poems)

and of Piping down the valleys wild, Piping songs of pleasant glee, On a cloud I saw a child, And he laughing said to me: "Pipe a song about a Lamb!" So I piped with merry cheer. "Piper, pipe that song again;" So I piped: he wept to hear. "Drop thy pipe, thy happy pipe; Sing thy songs of happy cheer:!" So I sang the same again, While he wept with joy to hear. "Piper, sit thee down and write In a book, that all may read." So he vanish'd from my sight; And I pluck'd a hollow reed, And I made a rural pen, And I stain'd the water clear, And I wrote my happy songs Every child may joy to hear. How sweet is the Shepherd's sweet lot! From the morn to the evening he stays; He shall follow his sheep all the day, And his tongue shall be filled with praise. For he hears the lambs' innocent call, And he hears the ewes' tender reply; He is watching while they are in peace, For they know when their Shepherd is nigh. The sun does arise, And make happy the skies; The merry bells ring To welcome the Spr

In [8]:
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)
emma_doc = nlp(emma)
paradise_doc = nlp(paradise)
parents_doc = nlp(parents)
ball_doc = nlp(ball)
mobydick_doc = nlp(mobydick)
busterbrown_doc = nlp(busterbrown)
stories_doc = nlp(stories)
poems_doc = nlp(poems)

In [9]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
paradise_sents = [[sent, "Milton"] for sent in paradise_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
parents_sents = [[sent, "Edgeworth"] for sent in parents_doc.sents]
busterbrown_sents = [[sent, "Burgess"] for sent in busterbrown_doc.sents]
mobydick_sents = [[sent, "Melville"] for sent in mobydick_doc.sents]
stories_sents = [[sent, "Bryant"] for sent in stories_doc.sents]
poems_sents = [[sent, "Blake"] for sent in poems_doc.sents]
ball_sents = [[sent, "Chesterton"] for sent in ball_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + paradise_sents + emma_sents + persuasion_sents + parents_sents + busterbrown_sents + mobydick_sents + stories_sents + poems_sents + ball_sents)
sentences.tail(50)

,0,1
39332,"(While, we, can, not, solicit, donations, from...",Chesterton
39333,"(International, donations, are, accepted, ,, b...",Chesterton
39334,"(Donations, by, check, or, money, order, may, ...",Chesterton
39335,"(Oxford, ,, 38655, -, 4109, Contact, us, if, y...",Chesterton
39336,"(The, Project, Gutenberg, Literary, Archive, F...",Chesterton
39337,"(Donations, are, tax, -, deductible, to, the, ...",Chesterton
39338,"(As, fund, -, raising, requirements, for, othe...",Chesterton
39339,"(We, need, your, donations, more, than, ever, !)",Chesterton
39340,"(You, can, get, up, to, date, donation, inform...",Chesterton
39341,"(Hart, will, answer, or, forward, your, messag...",Chesterton


In [10]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(5000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
paradisewords = bag_of_words(paradise_doc)
emmawords = bag_of_words(emma_doc)
persuasionwords = bag_of_words(persuasion_doc)
parentswords = bag_of_words(parents_doc)
busterbrownwords = bag_of_words(busterbrown_doc)
mobydickwords = bag_of_words(mobydick_doc)
storieswords = bag_of_words(stories_doc)
poemswords = bag_of_words(poems_doc)
ballwords = bag_of_words(ball_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + paradisewords + emmawords + persuasionwords + parentswords + busterbrownwords + mobydickwords + storieswords + poemswords + ballwords)

In [ ]:
print(common_words)

{'twould', 'sullenness', 'mac', 'replacement', "cur'ous", 'less', 'scorn', 'event', 'estrangement', 'moonshine', 'convention', 'outlive', 'select', 'goodnatur', "m'doole", 'ceiling', "know'st", 'hut', 'bathe', 'unbending', 'seedling', 'musky', 'barbarian', 'willingness', 'universally', 'aching', 'pequod', 'idolator', 'individual', 'beholder', 'jerk', 'wags', 'hydras', 'sympathize', 'felow', 'sheepish', 'scanty', 'preoccupied', 'burn', 'altiora', 'goody', 'ravenous', 'tellest', 'competitor', 'parsley', 'jewel', 'suffocate', 'adherence', 'shark', 'purple', 'superiour', 'fox', 'urge', 'gently', 'thanker', 'imaginist', 'fireplace', 'piteous', 'fricassee', 'false', 'fire', 'adamantine', 'osiris', 'sufferer', 'glistering', 'un', 'carefulness', 'feasible', 'superintend', 'traveller', 'park', 'brow', 'timorous', 'hungry', 'pirate', 'embarrass', 'sensibility', 'baked', 'misconstruction', 'unbutton', 'garnish', 'occupation', 'madness', 'motion', 'webster', 'complex', 'sanctities', 'gentlemanly',

In [ ]:
word_counts = bow_features(sentences, common_words)
word_counts.head()

In [92]:
word_counts.tail(100)

,blaze,recorder,prolong,obtain,imaus,terror,march,hesperus,trickle,calmly,...,fearlessness,rightly,misinformed,refuge,goliath,straight,unfeigned,29th,text_sentence,text_source
39282,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(All, his, life, up, to, that, moment, he, had...",Chesterton
39283,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(But, he, was, unlike, the, writers, in, the, ...",Chesterton
39284,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(As, the, little, singing, figure, came, neare...",Chesterton
39285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Then, the, little, old, man, went, past, them...",Chesterton
39286,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(They, had, not, looked, at, his, face, .)",Chesterton
39287,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(When, he, had, passed, they, looked, up, .)",Chesterton
39288,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(While, the, first, light, of, the, fire, had,...",Chesterton
39289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(But, now, the, fire, was, turned, to, left, a...",Chesterton
39290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(But, it, struck, something, that, was, neithe...",Chesterton
39291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Far, ,, far, away, up, in, those, huge, hollo...",Chesterton


In [93]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier(n_estimators=100, max_depth=15)
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.25,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.448503521127

Test set score: 0.438756855576


In [ ]:
svm = SVC(C=5)
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = svm.fit(X_train, y_train)

print('Training set score:', svm.score(X_train, y_train))
print('\nTest set score:', svm.score(X_test, y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

In [10]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.921806167401

Test set score: 0.908039647577


In [ ]:
emma=gutenberg.paras('austen-emma.txt')
#processing
emma_paras=[]
for paragraph in emma:
    para=paragraph[0]
    #removing the double-dash from all words
    para=[re.sub(r'--','',word) for word in para]
    #Forming each paragraph into a string and adding it to the list of strings.
    emma_paras.append(' '.join(para))

print(emma_paras[0:4])

In [ ]:


X_train, X_test = train_test_split(emma_paras, test_size=0.4, random_state=0)

vectorizer = TfidfVectorizer(max_df=0.25, # drop words that occur in more than half the paragraphs
                             min_df=5, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
emma_paras_tfidf=vectorizer.fit_transform(emma_paras)
print("Number of features: %d" % emma_paras_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf= train_test_split(emma_paras_tfidf, test_size=0.4, random_state=0)


#Reshapes the vectorizer output into something people can read
X_train_tfidf_csr = X_train_tfidf.tocsr()

#number of paragraphs
n = X_train_tfidf_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()
#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]

#Keep in mind that the log base 2 of 1 is 0, so a tf-idf score of 0 indicates that the word was present once in that sentence.
print('Original sentence:', X_train[5])
print('Tf_idf vector:', tfidf_bypara[5])

In [93]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier(n_estimators=100, max_depth=15)
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.25,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.448503521127

Test set score: 0.438756855576


In [ ]:
svm = SVC(C=5)
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = svm.fit(X_train, y_train)

print('Training set score:', svm.score(X_train, y_train))
print('\nTest set score:', svm.score(X_test, y_test))

In [94]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

max_iter reached after 858 seconds


/Users/kylehoward/Desktop/Thinkful/anaconda/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 14.3min remaining:    0.0s


max_iter reached after 414 seconds


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 21.2min remaining:    0.0s


max_iter reached after 412 seconds


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 28.1min remaining:    0.0s


KeyboardInterrupt: 

In [10]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.921806167401

Test set score: 0.908039647577
